In [2]:
from utility.utility_general import *
from pymatgen.ext.matproj import MPRester
from collections import Counter
# from predict_synthesis import *
import requests
from time import sleep
import tqdm
import crystals_tools
from IPython.display import display
from ase.formula import Formula
from ase.io import read as ase_read

electrode_path = f'{data_path}/electrode_materials'
assert exists(electrode_path)

In [2]:
api_key = "MCpqDZgh29W9r6X4Ru"

mpr: MPRester = MPRester(api_key)

def get_battery_data(self, formula_or_batt_id):
    """Returns batteries from a batt id or formula.

    Examples:
        get_battery("mp-300585433")
        get_battery("LiFePO4")
    """
    return self._make_request('/battery/%s' % formula_or_batt_id)

MPRester.get_battery_data = get_battery_data

## Getting electrode materials from the MP
### Getting a list of MP electrode materials 

In [12]:
battery_ids = mpr._make_request('/battery/all_ids')
df = pd.DataFrame({'id': battery_ids})
df['query'] = None
working_ion = [i.split('_')[-1] for i in battery_ids]
print(Counter(working_ion))
print('Total electrode ids: ', len(battery_ids))

Counter({'Li': 2291, 'Ca': 484, 'Mg': 393, 'Zn': 385, 'Na': 328, 'Y': 157, 'Al': 149, 'K': 125, 'Rb': 50, 'Cs': 39})
Total electrode ids:  4401


### Getting information from MP 

In [24]:
bat_left = df[df['query'].isna()]
print('Current number of electrode queries: {:,}'.format(len(df[~df['query'].isna()])))
for ind, row in tqdm.tqdm_notebook(bat_left.iterrows(), total=len(bat_left)):
    q = mpr.get_battery_data(row['id'])
    df.loc[ind, 'query'] = q
df['n_query'] = [len(i) for i in df['query']]
df.to_pickle(electrode_path + '/query_raw_electrode.pkl')
print('Found structures : {:,}'.format(len(df[~df['query'].isna()])))

### Extracting properties from each query

In [172]:
df = pd.read_pickle(electrode_path + '/query_raw_electrode.pkl')
for col in ['battid', 'working_ion', 'average_voltage', 'capacity_vol']:
    df[col] = [i[0][col] for i in df['query']]

for col in ['formula_charge', 'id_charge', 'formula_discharge', 'id_discharge']:
    df[col] = [i[0]['adj_pairs'][0][col] for i in df['query']]
    
df['sgn'] = [i[0]['spacegroup']['number'] for i in df['query']]
df['hall'] = [i[0]['spacegroup']['hall'] for i in df['query']]

assert np.all(df['id'] == df['battid'])

df.drop(columns=['id', 'query', 'n_query']).to_csv(electrode_path + '/mp_electrode_materials.csv')
df.drop(columns=['id', 'query', 'n_query'])

,battid,working_ion,average_voltage,capacity_vol,formula_charge,id_charge,formula_discharge,id_discharge,sgn,hall
0,mp-504791_Li,Li,2.325185,269.262771,P(WO4)2,mp-504791,LiP(WO4)2,mp-763526,19,P 2ac 2ab
1,mp-763480_Li,Li,3.291748,154.215698,P3W2O13,mp-767085,LiP3W2O13,mp-763539,14,-P 2ybc
2,mp-1176966_Li,Li,3.609671,651.708695,P8W3O29,mp-777230,Li2P8W3O29,mp-777867,165,"-P 3 2""c"
3,mvc-5592_Li,Li,3.201169,329.726946,P2WO7,mvc-5592,LiP2WO7,mp-763367,2,-P 1
4,mp-763566_Li,Li,2.438309,306.081406,P2WO7,mp-773366,LiP2WO7,mp-763566,4,P 2yb
...,...,...,...,...,...,...,...,...,...,...
4396,mp-1180276_Mg,Mg,-0.098693,461.340369,VCu3Se4,mp-21855,MgVCu3Se4,mp-1180276,215,P -4 2 3
4397,mp-1042619_Mg,Mg,1.890890,387.828816,Cu3(SnO3)4,mvc-13015,MgCu3(SnO3)4,mp-1042619,204,-I 2 2 3
4398,mp-1046932_Mg,Mg,3.071616,914.229317,BaTl(NiO3)2,mp-1046932,Ba2Mg3Tl2(NiO3)4,mvc-132,1,P 1
4399,mp-1041103_Mg,Mg,1.982244,735.558063,Ho(MoO3)2,mp-1041198,HoMg(MoO3)2,mp-1041103,1,P 1


### Getting the CIF files from the MP
#### Preparing the browser

In [187]:

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os, time

# download_path = os.path.expanduser('~/Downloads/test/')
download_path = data_path + '/electrode_materials/downloads_2'

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Default chrome download path
chrome_options.add_experimental_option("prefs", {
        "download.default_directory": f"{download_path}",
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing_for_trusted_sources_enabled": False,
        "safebrowsing.enabled": False
})

driver = webdriver.Chrome(os.path.expanduser('~/Apps/chromedriver'),
                          options=chrome_options)

#### First time log in 
Check your email after this and use the link in the following box

In [175]:
driver.get("http://www.materialsproject.org")
time.sleep(2)

elem = driver.find_element_by_css_selector('[class="register-block"]')
elem.click()
time.sleep(2)

elem = driver.find_element_by_css_selector('[class="btn btn-success"]')
elem.click()
time.sleep(2)

elem = driver.find_element_by_name('username')
elem.click()
elem.send_keys('a.davari871@gmail.com')
elem = driver.find_element_by_css_selector('[type="submit"]')
elem.click()
time.sleep(2)

In [176]:
# log in 
link = 'http://materialsproject.org/accounts/login/code/?code=7419a71dfa5b4898ef71'
driver.get(link)
time.sleep(2)
driver.save_screenshot("tmp/screenshot.png")
pickle.dump(driver.get_cookies() , open("tmp/cookies.pkl","wb"))
driver.close()
del driver

#### Testing the log in to the MP

In [179]:
# load login 
driver.get("http://www.materialsproject.org")
cookies = pickle.load(open("tmp/cookies.pkl", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)
    
driver.get("http://www.materialsproject.org")
# time.sleep(2)
driver.save_screenshot("tmp/screenshot.png")

True

#### Downloading the files

In [188]:
def download_cifs_from_mp(url_list):
    driver_f = webdriver.Chrome(os.path.expanduser('~/Apps/chromedriver'),
                          options=chrome_options)
    driver_f.get("http://www.materialsproject.org")
    cookies = pickle.load(open("tmp/cookies.pkl", "rb"))
    for cookie in cookies:
        driver_f.add_cookie(cookie)
        
    download = []
    for i in tqdm.tqdm_notebook(url_list, total=len(url_list)):
        try:
            driver_f.get(i)
            time.sleep(2)

            elem = driver_f.find_element_by_css_selector('[class="download-cif btn-group pull-right"]')
            elem.click()

            elem = driver_f.find_element_by_css_selector('[data-ciftype="computed"]')
            elem.click()
            time.sleep(2)
            download.append(u)
        except:
            download.append(None)
    driver_f.close()
    return download

In [189]:
# df['url'] = [f'https://materialsproject.org/batteries/{i}/' for i in df['id']]
df['url'] = [f'https://materialsproject.org/materials/{i}/' for i in df['id_discharge']]


from joblib import Parallel, delayed
url = df['url'].to_list()
iterable = np.array_split(url, 10)

output = Parallel(n_jobs=32)(delayed(download_cifs_from_mp)
                             (it) 
                             for it in tqdm.tqdm_notebook(iterable))

#### Checking and merging the CIF files

In [174]:
files = list_all_files(electrode_path + '/downloads_2/', pattern='*.cif')
download_df = pd.DataFrame({
    'id_discharge': [i.split('/')[-1].split('_')[-2] for i in files],
    'downloaded': True,
    'formula_downloaded': [i.split('/')[-1].split('_')[0] for i in files],
    'path': files,
#     'atoms': [crystals_tools.cif_parser(i) for i in files]
})
download_df = pd.merge(df, download_df, on='id_discharge', how='left').dropna()
download_df = download_df.drop_duplicates(['path']).reset_index()
download_df['min_atomic_dist'] = [crystals_tools.atoms_find_min_dist(crystals_tools.cif_parser(i))
                            for i in download_df['path']]

assert (download_df['min_atomic_dist'] > 0.5).all()
assert (download_df['formula_discharge'] == download_df['formula_downloaded']).all()
assert (download_df['battid'] == download_df['id']).all()
assert ~download_df.duplicated(['path']).any()

download_df = download_df.drop(columns=['downloaded', 'formula_downloaded', 'query', 'n_query', 'id'])
download_df.to_csv(electrode_path + '/mp_electrode_materials.csv', index=False)
download_df

,index,battid,working_ion,average_voltage,capacity_vol,formula_charge,id_charge,formula_discharge,id_discharge,sgn,hall,path,min_atomic_dist
0,10,mp-1176594_Li,Li,3.646267,188.723485,P4WO12,mp-767563,LiP4WO12,mp-1176594,15,-C 2yc,/home/adavar2/Data/syn/electrode_materials/dow...,1.467421
1,19,mp-764671_Li,Li,2.242338,197.124575,Mn7O7F,mp-767193,LiMn7O7F,mp-764671,156,"P 3 -2""",/home/adavar2/Data/syn/electrode_materials/dow...,1.903595
2,23,mp-540145_Li,Li,5.327179,295.684251,Fe(PO3)5,mp-540145,LiFe(PO3)5,mp-687200,11,-P 2yb,/home/adavar2/Data/syn/electrode_materials/dow...,1.483575
3,24,mp-1101552_Li,Li,5.484298,326.301416,FeP4O13,mp-1101552,LiFeP4O13,mp-705398,2,-P 1,/home/adavar2/Data/syn/electrode_materials/dow...,1.483159
4,25,mp-1177017_Li,Li,5.122810,460.648889,Li3Fe3P8O29,mp-705437,Li6Fe3P8O29,mp-1177017,1,P 1,/home/adavar2/Data/syn/electrode_materials/dow...,1.501046
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2086,4384,mvc-14217_Mg,Mg,3.308927,878.356628,MoF5,mvc-14217,MgMoF5,mvc-14220,1,P 1,/home/adavar2/Data/syn/electrode_materials/dow...,1.930655
2087,4393,mp-1096950_Mg,Mg,0.006906,3845.618655,Fe,mp-1096950,Mg149Fe,mp-1185592,191,-P 6 2,/home/adavar2/Data/syn/electrode_materials/dow...,2.985710
2088,4396,mp-1180276_Mg,Mg,-0.098693,461.340369,VCu3Se4,mp-21855,MgVCu3Se4,mp-1180276,215,P -4 2 3,/home/adavar2/Data/syn/electrode_materials/dow...,2.418948
2089,4398,mp-1046932_Mg,Mg,3.071616,914.229317,BaTl(NiO3)2,mp-1046932,Ba2Mg3Tl2(NiO3)4,mvc-132,1,P 1,/home/adavar2/Data/syn/electrode_materials/dow...,1.868003


### Putting the CIF files in the bank folder

In [238]:
makedirs(data_path + '/data_banks/electrode-mp/', exist_ok=True)
download_df.to_csv(data_path + '/data_banks/electrode-mp/mp_electrode_materials.csv', index=False)

for i, _ in download_df.iterrows():
    shutil.copy(download_df['path'][i], 
                data_path + '/data_banks/electrode-mp/' + download_df['path'][i].split('/')[-1])    

## Preparing a data set of COD electrode materials 
The preparation process is being done in the positive_data_preparation.ipynb

## Merging MP and COD

In [182]:
[i for i in list(mp_df.columns) if i in cod_df.columns]

['id',
 'working_ion',
 'capacity_vol',
 'formula_charge',
 'id_charge',
 'formula_discharge',
 'id_discharge',
 'sgn',
 'hall',
 'path',
 'min_atomic_dist',
 'Formula_hill',
 'db']

In [3]:
pd.read_csv(data_path + '/electrode_materials/cod-electrode-materials.csv', index_col=0)

,id,formula,db,filename,min_atomic_dist,Formula_hill,sgn,index,battid,working_ion,max_voltage,capacity_vol,formula_charge,id_charge,formula_discharge,id_discharge,hall,url,path
0,1000075,Bi2Li2O8Pd4,cod,/data_bases/cod/cif/1/00/00/1000075.cif,1.952114,BiLiO4Pd2,129,1535,mp-25201_Li,Li,4.138586,408.336004,Bi(PdO2)2,mp-25201,LiBi(PdO2)2,mp-559893,P 4ab 2ab -1ab,https://materialsproject.org/materials/mp-559893/,/home/adavar2/Data/syn/electrode_materials/dow...
1,1000268,Cu2La8Li2O16,cod,/data_bases/cod/cif/1/00/02/1000268.cif,1.786956,CuLa4LiO8,65,2178,mp-21496_Li,Li,3.933088,239.350906,La4CuO8,mp-772213,LiLa4CuO8,mp-21496,-C 2 2,https://materialsproject.org/materials/mp-21496/,/home/adavar2/Data/syn/electrode_materials/dow...
2,1001384,Na4O28P8Ti4,cod,/data_bases/cod/cif/1/00/13/1001384.cif,1.492221,NaO7P2Ti,14,3686,mp-560506_Na,Na,1.609686,315.744642,TiP2O7,mp-761010,NaTiP2O7,mp-560506,-P 2ybc,https://materialsproject.org/materials/mp-560506/,/home/adavar2/Data/syn/electrode_materials/dow...
3,1001384,Na4O28P8Ti4,cod,/data_bases/cod/cif/1/00/13/1001384.cif,1.492221,NaO7P2Ti,14,3687,mp-17461_Na,Na,1.764832,323.582741,TiP2O7,mp-761041,NaTiP2O7,mp-17461,-P 2ybc,https://materialsproject.org/materials/mp-17461/,/home/adavar2/Data/syn/electrode_materials/dow...
4,1001385,Na4O28P8Ti4,cod,/data_bases/cod/cif/1/00/13/1001385.cif,1.489720,NaO7P2Ti,14,3686,mp-560506_Na,Na,1.609686,315.744642,TiP2O7,mp-761010,NaTiP2O7,mp-560506,-P 2ybc,https://materialsproject.org/materials/mp-560506/,/home/adavar2/Data/syn/electrode_materials/dow...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,1544720,KMn8O16,cod-sc,/data_bases/cod/cif_supercell/1/54/47/1544720_...,1.886621,KMn8O16,1,2872,mp-1003316_K,K,4.272564,512.975679,MnO2,mp-19395,KMn8O16,mp-1016153,P 1,https://materialsproject.org/materials/mp-1016...,/home/adavar2/Data/syn/electrode_materials/dow...
278,1533548,Cu8Ga4O28Sr8Y4,cod-sc,/data_bases/cod/cif_supercell/1/53/35/1533548_...,1.613466,Cu2GaO7Sr2Y,1,3128,mvc-15173_Y,Y,3.487032,511.353349,Sr6YGa3(Cu2O7)3,mvc-15173,Sr2YGaCu2O7,mp-556575,P 1,https://materialsproject.org/materials/mp-556575/,/home/adavar2/Data/syn/electrode_materials/dow...
279,2018541,Fe4Li4O16P4,cod-sc,/data_bases/cod/cif_supercell/2/01/85/2018541_...,1.521004,FeLiO4P,1,62,mp-767263_Li,Li,3.692890,255.900638,LiFe2(PO4)2,mp-767263,LiFePO4,mp-774251,P 1,https://materialsproject.org/materials/mp-774251/,/home/adavar2/Data/syn/electrode_materials/dow...
280,4344259,Li8NbO6,cod-sc,/data_bases/cod/cif_supercell/4/34/42/4344259_...,1.768640,Li8NbO6,1,1806,mp-37399_Li,Li,0.162446,327.063919,Li7NbO6,mp-37399,Li8NbO6,mp-774697,P 1,https://materialsproject.org/materials/mp-774697/,/home/adavar2/Data/syn/electrode_materials/dow...


In [184]:
cod_df = pd.read_csv(data_path + '/electrode_materials/cod-electrode-materials.csv', index_col=0)
mp_df = pd.read_csv(electrode_path + '/mp_electrode_materials.csv')

cod_df = cod_df.drop_duplicates(subset=['filename'])
cod_df['filename'] = local_data_path + cod_df['filename']

mp_df.rename(columns={'battid': 'id'}, inplace=True)
del cod_df['path']
cod_df.rename(columns={'filename': 'path'}, inplace=True)
mp_df['Formula_hill'] = pd.Series([Formula(i).reduce()[0].format('hill') 
                                   for i in mp_df['formula_discharge']])
mp_df['db'] = 'mp'
# cod_df.drop(columns=['Formula_hill', 'formula'], inplace=True)
cod_df.drop(columns=['formula'], inplace=True)
mp_df.drop(columns=['index'], inplace=True)
mp_df.rename(columns={'battid': 'id'}, inplace=True)

# cod_df = cod_df[list(mp_df.columns)]
cod_df = cod_df[[i for i in list(mp_df.columns) if i in cod_df.columns]]
electrode_df = pd.concat([mp_df, cod_df], ignore_index=True)

# marking the mp samples available in the cod
df = electrode_df[electrode_df.duplicated(subset=['Formula_hill', 'sgn'], keep=False)]
electrode_df.loc[df[df['db']=='mp'].index, 'db'] = 'mp-avail-in-cod'
print(np.unique(electrode_df['db'], return_counts=True))

assert np.all([exists(i) for i in electrode_df['path']])

electrode_df.to_csv(data_path + '/electrode_materials/electrode-materials.csv', index=False)
electrode_df

(array(['cod', 'cod-sc', 'mp', 'mp-avail-in-cod'], dtype=object), array([ 258,    6, 1824,  267]))


,id,working_ion,average_voltage,capacity_vol,formula_charge,id_charge,formula_discharge,id_discharge,sgn,hall,path,min_atomic_dist,Formula_hill,db
0,mp-1176594_Li,Li,3.646267,188.723485,P4WO12,mp-767563,LiP4WO12,mp-1176594,15,-C 2yc,/home/adavar2/Data/syn/electrode_materials/dow...,1.467421,LiO12P4W,mp
1,mp-764671_Li,Li,2.242338,197.124575,Mn7O7F,mp-767193,LiMn7O7F,mp-764671,156,"P 3 -2""",/home/adavar2/Data/syn/electrode_materials/dow...,1.903595,FLiMn7O7,mp
2,mp-540145_Li,Li,5.327179,295.684251,Fe(PO3)5,mp-540145,LiFe(PO3)5,mp-687200,11,-P 2yb,/home/adavar2/Data/syn/electrode_materials/dow...,1.483575,FeLiO15P5,mp
3,mp-1101552_Li,Li,5.484298,326.301416,FeP4O13,mp-1101552,LiFeP4O13,mp-705398,2,-P 1,/home/adavar2/Data/syn/electrode_materials/dow...,1.483159,FeLiO13P4,mp
4,mp-1177017_Li,Li,5.122810,460.648889,Li3Fe3P8O29,mp-705437,Li6Fe3P8O29,mp-1177017,1,P 1,/home/adavar2/Data/syn/electrode_materials/dow...,1.501046,Fe3Li6O29P8,mp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2350,1544720,K,NaN,512.975679,MnO2,mp-19395,KMn8O16,mp-1016153,1,P 1,/home/adavar2/Local_Data/syn/data_bases/cod/ci...,1.886621,KMn8O16,cod-sc
2351,1533548,Y,NaN,511.353349,Sr6YGa3(Cu2O7)3,mvc-15173,Sr2YGaCu2O7,mp-556575,1,P 1,/home/adavar2/Local_Data/syn/data_bases/cod/ci...,1.613466,Cu2GaO7Sr2Y,cod-sc
2352,2018541,Li,NaN,255.900638,LiFe2(PO4)2,mp-767263,LiFePO4,mp-774251,1,P 1,/home/adavar2/Local_Data/syn/data_bases/cod/ci...,1.521004,FeLiO4P,cod-sc
2353,4344259,Li,NaN,327.063919,Li7NbO6,mp-37399,Li8NbO6,mp-774697,1,P 1,/home/adavar2/Local_Data/syn/data_bases/cod/ci...,1.768640,Li8NbO6,cod-sc


## Finding ICSD tags of MP crystals

### Preparing the previous information

In [185]:
electrode_df = pd.read_csv(data_path + '/electrode_materials/electrode-materials.csv')
files = electrode_df.loc[(electrode_df['db']== 'mp') | (electrode_df['db']== 'mp-avail-in-cod'), 'path']
atoms = [ase_read(i) for i in files]
atoms = [Formula(str(a.symbols)).format('hill') for a in atoms]
electrode_df['full_formula_hill'] = None
electrode_df.loc[(electrode_df['db']== 'mp') | (electrode_df['db']== 'mp-avail-in-cod')\
                 , 'full_formula_hill'] = atoms

### Getting information from MP API

In [19]:
data = [mpr.get_data(i) for i in tqdm.tqdm_notebook(electrode_df['formula_discharge'].unique())]
save_var(data, 'tmp/mp-formula_discharge_query.pkl')

In [186]:
data = load_var('tmp/mp-formula_discharge_query.pkl')
data = [item for sublist in data for item in sublist]

df = pd.DataFrame({'query': data})

for col in ['material_id', 'pretty_formula', 'full_formula', 'task_ids', 'icsd_id', 'icsd_ids']:
    df[col] = [i[col] for i in df['query']]
df['icsd_ids'] = [(None if len(i)==0 else i) for i in df['icsd_ids']]
# for col in ['formula_charge', 'id_charge', 'formula_discharge', 'id_discharge']:
#     df[col] = [i[0]['adj_pairs'][0][col] for i in df['query']]
    
df['sgn'] = [i['spacegroup']['number'] for i in df['query']]
df['hall'] = [i['spacegroup']['hall'] for i in df['query']]
df['full_formula_hill'] = [Formula(i).format('hill') for i in df['full_formula']]

df = df[df['icsd_ids'].notna()]
df = df.drop_duplicates(subset=['sgn', 'full_formula_hill'], keep='first')
assert len(df[df.duplicated(subset=['sgn', 'full_formula_hill'], keep=False)]) == 0

### Combining the information

In [187]:
electrode_df.drop(columns=['icsd_ids'], errors='ignore', inplace=True)

electrode_df = pd.merge(electrode_df, df[['icsd_ids', 'sgn', 'full_formula_hill']],
             on=['sgn', 'full_formula_hill'], how='left')
# df = electrode_df
print('Total found structures with icsd: ', np.count_nonzero(electrode_df['icsd_ids'].notna()), '\n Of which:')
print(electrode_df[electrode_df['icsd_ids'].notna()]['db'].value_counts())

electrode_df.to_csv(data_path + '/electrode_materials/electrode-materials.csv', index=False)
electrode_df[electrode_df['icsd_ids'].notna()]

Total found structures with icsd:  212 
 Of which:
mp-avail-in-cod    114
mp                  98
Name: db, dtype: int64


,id,working_ion,average_voltage,capacity_vol,formula_charge,id_charge,formula_discharge,id_discharge,sgn,hall,path,min_atomic_dist,Formula_hill,db,full_formula_hill,icsd_ids
59,mp-1101258_Li,Li,1.991242,203.446131,TiS2,mp-1101258,Li(TiS2)3,mp-19755,164,"-P 3 2""",/home/adavar2/Data/syn/electrode_materials/dow...,2.413944,LiS6Ti3,mp,LiS6Ti3,"[31342, 60015]"
66,mp-543104_Li,Li,3.103617,440.462549,V2O5,mp-543104,LiV2O5,mp-627993,62,-P 2ac 2n,/home/adavar2/Data/syn/electrode_materials/dow...,1.647047,LiO5V2,mp-avail-in-cod,Li4O20V8,"[23817, 9129, 27245, 25384]"
88,mp-19017_Li,Li,3.404584,588.739457,FePO4,mp-25001,LiFePO4,mp-19017,62,-P 2ac 2n,/home/adavar2/Data/syn/electrode_materials/dow...,1.538153,FeLiO4P,mp-avail-in-cod,Fe4Li4O16P4,"[193799, 193800, 194331, 193797, 260570, 16681..."
95,mp-759435_Li,Li,3.569443,507.873226,Cu(PO3)3,mp-766254,LiCu(PO3)3,mp-759435,19,P 2ac 2ab,/home/adavar2/Data/syn/electrode_materials/dow...,1.487720,CuLiO9P3,mp-avail-in-cod,Cu4Li4O36P12,[2808]
112,mp-776882_Li,Li,3.619855,451.970861,V2O5,mp-776882,LiV2O5,mp-777667,62,-P 2ac 2n,/home/adavar2/Data/syn/electrode_materials/dow...,1.689952,LiO5V2,mp-avail-in-cod,Li4O20V8,"[23817, 9129, 27245, 25384]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2047,mvc-11142_Mg,Mg,5.197225,836.024423,CrF6,mvc-11142,MgCrF6,mp-554955,148,-R 3,/home/adavar2/Data/syn/electrode_materials/dow...,2.010248,CrF6Mg,mp-avail-in-cod,CrF6Mg,[10344]
2053,mp-1191852_Mg,Mg,0.758716,426.253201,Mg6TiH16,mp-1191852,Mg7TiH16,mp-644281,225,-F 4 2 3,/home/adavar2/Data/syn/electrode_materials/dow...,1.948356,H16Mg7Ti,mp,H16Mg7Ti,"[169050, 168839, 247913, 152886, 164595, 166233]"
2062,mvc-14468_Mg,Mg,4.207026,363.742211,MgV6(FeO6)4,mvc-14468,Mg3V6(FeO6)4,mp-19067,2,-P 1,/home/adavar2/Data/syn/electrode_materials/dow...,1.673142,Fe4Mg3O24V6,mp,Fe4Mg3O24V6,"[157616, 157615]"
2072,mp-1079163_Mg,Mg,-0.045083,3678.849568,Nb,mp-8636,Mg7Nb,mp-1079163,225,-F 4 2 3,/home/adavar2/Data/syn/electrode_materials/dow...,3.105153,Mg7Nb,mp,Mg7Nb,[164600]
